<a href="https://colab.research.google.com/github/samtickle808/Sentiment_classifier/blob/main/2_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

#choose input folder
input = '/content/drive/MyDrive/Sentiment Analyser/'

Mounted at /content/drive


In [6]:
#!pip install -q tensorflow-text
!pip install transformers
!pip install -q tf-models-official

import os
import shutil
import numpy as np
import pandas as pd

import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')




In [3]:
####choose models, weights, tokenizer

#for DistilBert
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

#for Bert
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [4]:
train = pd.read_csv(input + 'train.csv')

In [8]:
train = pd.read_csv(input + 'train.csv')
test = pd.read_csv(input + 'test.csv')
validation  = pd.read_csv(input + 'validation.csv')

def bert_prep(data, num_load = None, truncate = None):
  """function to prep dataset to be ready to use for BERT
  optional arguments are num_loan, which selects the first n rows for quicker testing
  truncate, additional argument that truncates sentences, again for quicker testing of functionality"""

#Tokenize
#reduce data loan to speed things up
  num_load = num_load
  tokenized = data['Text'][0:num_load].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

#truncate tokenized for now to get model to run
  tokenized = pd.Series([x[0:truncate] for x in tokenized])

  #Padding - this is needed to make sure all vectors are of same length
  max_len = max(tokenized.apply(lambda x: len(x)))
  padded = np.array([x + [0]*(max_len - len(x)) for x in tokenized.values])

#attention masking - we create vector telling the model to ignore the padding that we have added and not include it in calcs
  attention_mask = np.where(padded != 0, 1, 0)
  attention_mask.shape

  #save padded and attention_mask for output
  output = [padded, attention_mask]
  return(output)

bert_train, train_pad = bert_prep(train, num_load = 1000, truncate = 50)



In [ ]:
#Run model!

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)


with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

